In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from lineartree import LinearTreeRegressor
from sklearn.svm import LinearSVR
from sklearn import neighbors
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import *
import joblib
from collections import defaultdict
# import statsmodels.api as sm
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from pymongo import MongoClient
import warnings
from scipy import signal
import datetime
import csv
import os

from casadi import *

from IPython.display import clear_output

import datetime
warnings.filterwarnings(action='ignore')

import do_mpc

In [ ]:
#     0: '0', # 전원 OFF
#     1: '1', # 송풍모드
#     2: '2', # 가온모드
#     3: '3', # 송풍+제습
#     4: '4', # 냉각
#     5: '5', # 냉각+제습
#     6: '6', # 가온+제습

# Data read 
df_me = pd.read_feather('ecm_input.ftr')

In [ ]:
col_list = [
    'f3_temp',
    'f10_temp','extemp',
    'entemp',
    'F_SET_MODE_0'
]

df_datetime = df_me['timestamp_min']
df_pre = df_me[col_list]
df_pre.index = df_datetime

df_pre['running'] = 1
df_pre['running'].loc[df_pre['F_SET_MODE_0'] == 1] = 0
df_pre.drop(['F_SET_MODE_0'],axis = 'columns', inplace = True)
df_pre=df_pre.reindex(columns=['f3_temp','f10_temp','extemp','running','entemp'])
df_pre

In [ ]:
def MPC_StateSpace_parmeter(lr_model):

    A = np.array([lr_model.estimators_[0].coef_[0:3],
                  lr_model.estimators_[1].coef_[0:3],
                  lr_model.estimators_[2].coef_[0:3],
                  ]
                 )

    B_u = np.array([lr_model.estimators_[0].coef_[3],
                    lr_model.estimators_[1].coef_[3],
                    lr_model.estimators_[2].coef_[3],
                    ])

    B_d = np.array([lr_model.estimators_[0].coef_[4],
                    lr_model.estimators_[1].coef_[4],
                    lr_model.estimators_[2].coef_[4],
                  ])
    
    I_x = np.array([lr_model.estimators_[0].intercept_,
                    lr_model.estimators_[1].intercept_,
                    lr_model.estimators_[2].intercept_,
                  ])
    
    return A, B_u, B_d, I_x

In [ ]:
# Linear Regressor 모델 생성

#train 시작과 종료 일자
start_date = datetime.date(2019,8,24)
end_date = datetime.date(2019,10,1)

df_train = df_pre[((pd.to_datetime(df_pre.index).date) >= (start_date)) & 
           ((pd.to_datetime(df_pre.index).date) < (end_date))]

# 모델 학습
Shift_Target = 10 # 10분 뒤에 학습함
model = MultiOutputRegressor(LinearRegression())
model.fit(df_train[:-Shift_Target], df_train[Shift_Target:]) # 학습 시 일부데이터는 학습 하지 않음

# 모델 저장
filename = "MPC_LR_Coef_.joblib"
joblib.dump(model, filename)
print(f"{filename} 에 ML 모델이 저장되었습니다.")


In [ ]:
# LR 모델 불려오는 경우
path = 'MPC_LR_Coef_.joblib'
lr_model = joblib.load(path)

In [ ]:
# 테스트
start_date = datetime.date(2019,8,23)
end_date = datetime.date(2019,10,3)

# 날짜 array 생성
if (end_date-start_date).days == 0:
    end_date = start_date + datetime.timedelta(days=1)

Train_date_list = [start_date + datetime.timedelta(days=x) for x in range(0, (end_date-start_date).days)]

for i in range(0, len(Train_date_list)-2):
    
    # clear output
    clear_output(wait=True)
    
    Train_date = Train_date_list[i]
    Train_date_tvp = Train_date_list[i+1]
    print(Train_date)
    
    df_test_pre = df_pre[(pd.to_datetime(df_pre.index).date) == (Train_date)]
    df_test_pre_tvp = df_pre[(pd.to_datetime(df_pre.index).date) == (Train_date_tvp)]
    df_test_pre_tvp = pd.concat([df_test_pre,df_test_pre_tvp])
    
    # MPC 모델 생성
    model_type = 'discrete' # either 'discrete' or 'continuous'
    mpc_model = do_mpc.model.Model(model_type)

    # Inner temperature
    _x = mpc_model.set_variable(var_type='_x', var_name='x', shape=(3,1))
    # Control Input
    _u = mpc_model.set_variable(var_type='_u', var_name='u', shape=(1,1))
    # External temperature and External Humidity
    _d = mpc_model.set_variable(var_type='_tvp', var_name='d', shape=(1,1))

    # Set-point for the temperature
    temperature_set_point =mpc_model.set_variable(var_type='_tvp', var_name='temperature_set_point', shape=(1,1))

    A, B_u, B_d, I_x =MPC_StateSpace_parmeter(lr_model)

    x_next = A@_x+B_u@fmod(_u*10,1)+B_d@_d+I_x
    mpc_model.set_rhs('x', x_next)

    mpc_model.set_expression(expr_name='cost', expr=(((_x[0]-temperature_set_point)**2)**0.5))
    
    # Build the model
    mpc_model.setup()
    mpc = do_mpc.controller.MPC(mpc_model)

    setup_mpc = {
    'n_robust': 1,
    'n_horizon': 3,
    't_step': 1,
    'state_discretization': 'discrete',
    'store_full_solution':True,
    # Use MA27 linear solver in ipopt for faster calculations:
    #'nlpsol_opts': {'ipopt.linear_solver': 'MA27'}
    }
    mpc.set_param(**setup_mpc)

    # Cost
    mterm = mpc_model.aux['cost'] # terminal cost
    lterm = mpc_model.aux['cost'] # terminal cost
    # stage cost
    mpc.set_objective(mterm=mterm, lterm=lterm)

    #mpc.set_rterm(u=1e-4) # input penalty
    
    # Constraint setting
    max_x = np.array([[35.0], [35.0], [100.0]])
    min_x = np.array([[0.0], [0.0], [0.0]])

    # lower bounds of the states
    mpc.bounds['lower','_x','x'] = min_x
    # upper bounds of the states
    mpc.bounds['upper','_x','x'] = max_x
    # lower bounds of the input
    mpc.bounds['lower','_u','u'] = -0.001
    # upper bounds of the input
    mpc.bounds['upper','_u','u'] = 1.001

    # Model scaling
    mpc.scaling['_u','u'] = 1
    

    tvp_template = mpc.get_tvp_template()
 
    def tvp_fun(t_now):

        t_now = int(t_now)
        n_horizon = 3 
        for k in range(n_horizon+1):
            tvp_template['_tvp',k,'temperature_set_point'] = 25
            tvp_template['_tvp',k,'d'] =  np.array([df_test_pre_tvp['entemp'][t_now+k]])

        return tvp_template
    mpc.set_tvp_fun(tvp_fun)
    
    mpc.setup()
    
    # estimator 설정
    estimator = do_mpc.estimator.StateFeedback(mpc_model)
    
    simulator = do_mpc.simulator.Simulator(mpc_model)
    simulator.set_param(t_step = 1)
    tvp_template_sim = simulator.get_tvp_template()
 
    def tvp_fun_sim(t_now):

        t_now = int(t_now)
        tvp_template_sim['d'] = np.array([df_test_pre_tvp['entemp'][t_now]])

        return tvp_template_sim

    simulator.set_tvp_fun(tvp_fun_sim)
    
    # Quickly reset the history of the MPC data object.
    #mpc.reset_history()
    
    simulator.setup()
     
    # 초기값 셋팅
    x0 = np.array(df_test_pre.iloc[0][0:3])
    mpc.x0 = x0

    # mpc.set_initial_guess()

    for k in range(len(df_test_pre)):
        u0 = mpc.make_step(x0)
        pre_x0 = pd.DataFrame([{
            'f3_temp': x0[0],
            'f10_temp': x0[1],
            'exttemp' : x0[2],
            'running' : np.round(u0[0][0]),
            'entemp' : df_test_pre_tvp['entemp'][k+1]
        }])
        x0_output = model.predict(pre_x0)
        x0 = np.array(x0_output[0][0:3])
        

    # 내용 저장
    # 실제 운영된 것과 
    df_result = pd.DataFrame()
    df_result['timestamp'] = df_test_pre.index
    df_result['actural_x'] = pd.DataFrame(df_test_pre['f3_temp'].to_numpy())
    df_result['running'] = pd.DataFrame(df_test_pre['running'].to_numpy())
    df_result['mpc_x'] = pd.DataFrame(mpc.data['_x'][:,0])
    df_result['mpc_u'] = pd.DataFrame(np.round(mpc.data['_u']))
    
    df_result.to_csv('MPC_MPCSimulation_result_#1.csv',header =None, mode = 'a')
    
    


In [ ]:
from matplotlib import rcParams
rcParams['axes.grid'] = True
rcParams['font.size'] = 18

In [ ]:
import matplotlib.pyplot as plt
fig, ax, graphics = do_mpc.graphics.default_plot(mpc.data, figsize=(10,5))
graphics.plot_results()
graphics.reset_axes()
plt.show()

In [ ]:
#plt.plot(mpc.data['_x'][:,0])
plt.plot(np.round(mpc.data['_u']))
plt.show()

In [ ]:
# 컬렁명 : 0:index // 1:timestamp // 2:actural_x // 3:realrunning // 4:mpc_x // 5:mpc_control
# 에너지 절감율
df_exp= pd.read_csv('MPC_MPCSimulation_result_#1.csv', header = None)
(1-df_exp[5].sum()/df_exp[3].sum()) * 100